# HuggingFace: Summary of the Task:: MLM [link](https://huggingface.co/transformers/task_summary.html?highlight=fill%20mask#masked-language-modeling)

In [1]:
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
import numpy as np
from pathlib import Path
import pickle

In [2]:
class CustomSeniorProjectTokenizer(object):
    def __init__(self, TOK_PATH = Path('./senior_proj_itos'), BOS='xxbos', EOS='xxeos', FLD = 'xxfld', UNK='xxunk', PAD='xxpad',
                 TK_REP='xxrep', TK_WREP='xxwrep', TK_NUM='xxnum', TK_LAUGH='xxlaugh', n_cpus=1,
                ):
        from senior_project_util import ThaiTokenizer, pre_rules_th, post_rules_th
        from fastai.text.transform import BaseTokenizer, Tokenizer, Vocab
        from fastai.text.data import TokenizeProcessor, NumericalizeProcessor

        with open(TOK_PATH/"bert_itos_80k_cleaned.pkl", 'rb') as f:
            itos = pickle.load(f)
            
        self.vocab = Vocab(itos)
        self.tokenizer = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', 
                                   pre_rules = pre_rules_th, post_rules=post_rules_th, n_cpus=n_cpus)
        
        self.cls_token_id = self.vocab.stoi[BOS]
        self.sep_token_id = self.vocab.stoi[EOS]
        self.pad_token_id = self.vocab.stoi[PAD]
        
        self.mask_token = FLD  #SINCE THIS ONE IS NOT USED, and INSIDE SPECIAL TOKEN....
        self._pad_token = PAD
        
        self.mask_token_id = self.vocab.stoi[self.mask_token]
        
#         tokenizer_processor = TokenizeProcessor(tokenizer=tt, chunksize=300000, mark_fields=False)
#         numbericalize_processor = NumericalizeProcessor(vocab=vocab)
        
    def num_special_tokens_to_add(self, pair=False):
        return 2
    def tokenize(self, text):
        return self.tokenizer._process_all_1([text])[0]
#         return self.tokenizer.process_all([text])[0]
    
    def convert_tokens_to_ids(self, token_list):
        #From https://huggingface.co/transformers/_modules/transformers/tokenization_utils_fast.html#PreTrainedTokenizerFast.convert_tokens_to_ids
        if token_list is None:
            return None

        if isinstance(token_list, str):
            return self.vocab.numericalize([token_list])[0]
        
        return self.vocab.numericalize(token_list)
    
    def build_inputs_with_special_tokens(self, token_list):
        # From https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_bert.py#L235
        return [self.cls_token_id] + token_list + [self.sep_token_id]
    
    def get_special_tokens_mask(
        self, token_ids_0, token_ids_1 = None, already_has_special_tokens = False
    ):
        # From https://huggingface.co/transformers/_modules/transformers/tokenization_utils.html#PreTrainedTokenizer.get_special_tokens_mask
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` method.

        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model

        Returns:
            A list of integers in the range [0, 1]: 1 for a special token, 0 for a sequence token.
        """
        return [0] * ((len(token_ids_1) if token_ids_1 else 0) + len(token_ids_0))
    
    def __len__(self):
        #https://huggingface.co/transformers/_modules/transformers/tokenization_utils_fast.html#PreTrainedTokenizerFast.__len__
        return len(self.vocab.itos)
    
    def encode(self, text):
        _input = self.tokenize(text)
        _input = self.convert_tokens_to_ids(_input)
        _input = self.build_inputs_with_special_tokens(_input)
        _input = torch.tensor([_input])
        return _input
    def decode(self, numerical_token):
        if isinstance(numerical_token, list):
            return self.vocab.itos[numerical_token[0]]
        return self.vocab.itos[numerical_token]
    
tokenizer = CustomSeniorProjectTokenizer()

In [3]:
text = "ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ  ก่อนอื่นผมต้องบอกก่อนเลยว่าคนเราจะเลือกกินอาหารแบบไหนชอบแบบไหนเป็นเรื่องของความชอบส่วนตัวนะครับทุกคนมีสิทธิในการเลือกของที่ชอบและไม่ชอบอยู่แล้ว แต่ผมรู้สึกว่าตอนนี้ผมกำลังประสบปัญหาที่ดูเหมือนจะเล็กแต่กลายเป็นว่ามันค่อนข้างใหญ่ ผมคบกับแฟนมา6ปีแล้วครับ ผมเป็นคนชอบกินอาหารญี่ปุ่นและปลาดิบแต่แฟนผมไม่กินปลาดิบเลย ผมอยากกินบุฟเฟ่เนื้อแต่แฟนผมก็ไม่กินเนื้อ เราเลยไม่ได้เข้าทานร้านบุฟเฟ่เนื้อและบุฟเฟ่อาหารญี่ปุ่นกันเพราะรู้สึกลัวแฟนผมทานไม่คุ้ม และเรื่องใหญ่เลยคือผมเป็นคนชอบทานอาหารรสจัดและรสเผ็ดมาก แต่แฟนผมทานเผ็ดไม่ได้เลยเวลาเราไปกินส้มตำกันก็จะสั่ง ส้มตำไม่ใส่พริก ต้มแซ่บไม่ใส่พริก ลาบไม่ใส่พริก ร้านกับข้าวอื่นๆก็เช่นกันแฟนผมจะไม่ชอบกินผักไม่ค่อยสั่งกับข้าวที่เป็นผักแล้วผมชอบผักบุ้งทอดกรอบ เห็ดหอมสดทอดมาก แต่ก็ไม่ได้สั่งเพราะว่าเธอไม่กินถึงเค้าจะบอกให้สั่งเลยๆก็เถอะแต่ผมก็ยังเกรงใจเธออยู่ดีอ่ะครับ ผมรู้สึกกินอาหารไม่มีความสุขเลยชีวิตผมขาดรสเผ็ดไปเหมือนจะขาดใจเหมือนมันทำให้ขาดความสุขไปอย่างนึงเลยอ่ะครับ ยิ่งถ้าเราแต่งงานกันแล้วผมก็อาจจะต้องมีปัญหาเรื่องนี้มากขึ้น พอผมเห็นคู่ที่ชอบทานอาหารเหมือนๆกันเห็นเค้ากินอาหารกันอย่างมีความสุขแล้วผมรู้สึกอิจฉามากๆเลย มีใครเคยมีปัญหาแบบผมมั้ยครับแล้วจะแก้ปัญหานี้ยังไงดีครับ"
value = tokenizer.tokenize(text)
print(value)

['ใคร', 'เคย', 'มี', 'แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'แล้ว', 'รู้สึก', 'เสีย', 'ความสุข', 'ไป', 'อย่าง', 'นึง', 'บ้าง', 'มั้ย', 'ครับ', ' ', 'ก่อนอื่น', 'ผม', 'ต้อง', 'บอก', 'ก่อน', 'เลย', 'ว่า', 'คนเรา', 'จะ', 'เลือก', 'กิน', 'อาหาร', 'แบบ', 'ไหน', 'ชอบ', 'แบบ', 'ไหน', 'เป็นเรื่อง', 'ของ', 'ความชอบ', 'ส่วนตัว', 'นะ', 'ครับ', 'ทุกคน', 'มี', 'สิทธิ', 'ใน', 'การเลือก', 'ของที่ชอบ', 'และ', 'ไม่ชอบ', 'อยู่แล้ว', ' ', 'แต่', 'ผม', 'รู้สึก', 'ว่า', 'ตอนนี้', 'ผม', 'กำลัง', 'ประสบปัญหา', 'ที่', 'ดูเหมือน', 'จะ', 'เล็ก', 'แต่', 'กลายเป็น', 'ว่า', 'มัน', 'ค่อนข้าง', 'ใหญ่', ' ', 'ผม', 'คบ', 'กับ', 'แฟน', 'มา', 'xxnum', ' ', 'ปี', 'แล้ว', 'ครับ', ' ', 'ผม', 'เป็น', 'คน', 'ชอบ', 'กิน', 'อาหาร', 'ญี่ปุ่น', 'และ', 'ปลาดิบ', 'แต่', 'แฟน', 'ผม', 'ไม่', 'กิน', 'ปลาดิบ', 'เลย', ' ', 'ผม', 'อยากกิน', 'บุ', 'ฟเฟ่', 'เนื้อ', 'แต่', 'แฟน', 'ผม', 'ก็', 'ไม่', 'กิน', 'เนื้อ', ' ', 'เรา', 'เลย', 'ไม่ได้', 'เข้า', 'ทาน', 'ร้าน', 'บุ', 'ฟเฟ่', 'เนื้อ', 'และ', 'บุ', 'ฟเฟ่', 'อาหาร', 'ญี่ปุ่น', 

In [4]:
model = AutoModelWithLMHead.from_pretrained("./OriginalBert3_export")

/opt/conda/lib/python3.7/site-packages/transformers/modeling_auto.py:765: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [5]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(80000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [6]:
model.num_parameters()

148153472

In [11]:
nlp = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [12]:
nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks.")

AttributeError: 'CustomSeniorProjectTokenizer' object has no attribute 'batch_encode_plus'

In [13]:
nlp(f"วันนี้เป็นวันที่{nlp.tokenizer.mask_token}")

AttributeError: 'CustomSeniorProjectTokenizer' object has no attribute 'batch_encode_plus'

## HuggingFace Trainer [link](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py#L375)

## DataCollator [link](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L69)

1. A torch.utils.data.dataloader.Dataloader is created from a Dataset (where the examples are tokenized with `tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))` and appended special tokens `<s>` and `</s>` through `tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + self.block_size])` [this is created where `block_size` is `self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)`] ) referenced [here](https://github.com/huggingface/transformers/blob/9022ef021a56db975d25c7108cbd19d0dd399174/src/transformers/trainer.py#L224).  
This will return a "List" of ids. 
2. [_tensorize_batch()](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L90) will attempt to convert from ```examples: List[torch.Tensor]) -> torch.Tensor:``` and also apply [self.tokenizer._pad_token()](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L101) along the way for examples with irregular length
3. ```if self.mlm: ``` , mask the tokens with [mask_tokens()](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L103) to Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.

In [7]:
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."

In [8]:
_input = tokenizer.encode(sequence)
print(list(map(lambda x: tokenizer.decode([x]) , _input[0])))
_input

['xxbos', 'xxunk', ' ', 'models', ' ', 'are', ' ', 'xxunk', ' ', 'than', ' ', 'the', ' ', 'models', ' ', 'they', ' ', 'mimic', '.', ' ', 'using', ' ', 'them', ' ', 'instead', ' ', 'of', ' ', 'the', ' ', 'large', ' ', 'versions', ' ', 'would', ' ', 'help', ' ', 'xxfld', ' ', 'our', ' ', 'carbon', ' ', 'footprint', '.', 'xxeos']


tensor([[    2,     0,     9, 50708,     9,  5400,     9,     0,     9, 15262,
             9,   379,     9, 50708,     9, 11009,     9, 76615,    47,     9,
         29179,     9, 17114,     9, 54855,     9,   882,     9,   379,     9,
         14533,     9, 73189,     9, 15472,     9, 12358,     9,     4,     9,
          6858,     9, 17364,     9, 77218,    47,     3]])

In [9]:
labels = _input.clone()
# We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
probability_matrix = torch.full(labels.shape, 0.15)
probability_matrix

tensor([[0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500]])

In [10]:
# If there is special tokens, fill it with 0.0
special_tokens_mask = [
    tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
]
print(special_tokens_mask)
probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
probability_matrix

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


tensor([[0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500]])

In [11]:
# IF there is padding mask, fill it with 0 too
padding_mask = labels.eq(tokenizer.pad_token_id)
probability_matrix.masked_fill_(padding_mask, value=0.0)
padding_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False]])

In [12]:
# Randomly select which one to mask based on the probabilities given ealier, and if the mask isn't chosen, fill in -100 for the value
masked_indices = torch.bernoulli(probability_matrix).bool()
print(masked_indices)
labels[~masked_indices] = -100  # We only compute loss on masked 
labels

tensor([[ True, False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True, False,  True,  True, False,  True, False, False,  True,
         False,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False]])


tensor([[    2,  -100,  -100,  -100,     9,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,     9,  -100,     9, 54855,  -100,   882,  -100,  -100,     9,
          -100,     9,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100]])

In [13]:
# 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
_input[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

# 10% of the time, we replace masked input tokens with random word
indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
_input[indices_random] = random_words[indices_random]

# The rest of the time (10% of the time) we keep the masked input tokens unchanged
print(_input, labels)

tensor([[    4,     0,     9, 50708,     4,  5400,     9,     0,     9, 15262,
             9,   379,     9, 50708,     9, 11009,     9, 76615,    47,     9,
         29179,     9, 17114,     4,     4,     9,     4,     9,   379,     4,
         14533,     4, 73189,     9, 15472,     9, 12358,     9,     4,     9,
          6858,     9, 17364,     9, 77218,    47,     3]]) tensor([[    2,  -100,  -100,  -100,     9,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,     9,  -100,     9, 54855,  -100,   882,  -100,  -100,     9,
          -100,     9,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100]])


In [14]:
print(list(map(lambda x: tokenizer.decode([x]) , _input[0])))

['xxfld', 'xxunk', ' ', 'models', 'xxfld', 'are', ' ', 'xxunk', ' ', 'than', ' ', 'the', ' ', 'models', ' ', 'they', ' ', 'mimic', '.', ' ', 'using', ' ', 'them', 'xxfld', 'xxfld', ' ', 'xxfld', ' ', 'the', 'xxfld', 'large', 'xxfld', 'versions', ' ', 'would', ' ', 'help', ' ', 'xxfld', ' ', 'our', ' ', 'carbon', ' ', 'footprint', '.', 'xxeos']


# Now I'm going to define `_mask_input()` 
 0.15 Probability: With 100% replace with mask, and 0% unchanged

In [15]:
def _mask_input(sequence, tokenizer, mlm_probability=0.15, truncation=True):
    # Tokenize the sequence with special tokens inserted
    _input = tokenizer.encode(sequence)
    
    labels = _input.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    
    # If there is special tokens, fill it with 0.0
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    
    # IF there is padding mask, fill it with 0 too
    padding_mask = labels.eq(tokenizer.pad_token_id)
    probability_matrix.masked_fill_(padding_mask, value=0.0)
    
    # Randomly select which one to mask based on the probabilities given ealier, and if the mask isn't chosen, fill in -100 for the value
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked 
    
    # 100% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = masked_indices
    _input[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
    
    return _input, labels

In [16]:
tokenized_input, labels = _mask_input(f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help our carbon footprint.", tokenizer)
print(tokenized_input)
print(list(map(lambda x: tokenizer.decode([x]) , tokenized_input[0])))
print(labels)
print(list(map(lambda x: tokenizer.decode([x]) if x != -100 else '-', labels[0])))

tensor([[    2,     0,     9, 50708,     9,     4,     9,     4,     9,     4,
             9,   379,     9, 50708,     4, 11009,     9, 76615,     4,     9,
             4,     9, 17114,     9, 54855,     9,   882,     9,   379,     9,
             4,     9, 73189,     9, 15472,     4, 12358,     9,  6858,     9,
             4,     4,     4,    47,     3]])
['xxbos', 'xxunk', ' ', 'models', ' ', 'xxfld', ' ', 'xxfld', ' ', 'xxfld', ' ', 'the', ' ', 'models', 'xxfld', 'they', ' ', 'mimic', 'xxfld', ' ', 'xxfld', ' ', 'them', ' ', 'instead', ' ', 'of', ' ', 'the', ' ', 'xxfld', ' ', 'versions', ' ', 'would', 'xxfld', 'help', ' ', 'our', ' ', 'xxfld', 'xxfld', 'xxfld', '.', 'xxeos']
tensor([[ -100,  -100,  -100,  -100,  -100,  5400,  -100,     0,  -100, 15262,
          -100,  -100,  -100,  -100,     9,  -100,  -100,  -100,    47,  -100,
         29179,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         14533,  -100,  -100,  -100,  -100,     9,  -100,  -100,  -100, 

# Now I'm going to define `_predict_masks()`

In [50]:
def _predict_masks(_input, labels, tokenizer, k=5, verbose=True, verbose_length=6):
    mask_token_index = torch.where(_input == tokenizer.mask_token_id)[1]
#     print(mask_token_index)
    token_logits = model(_input)[0]
#     print(token_logits)
    mask_token_logits = token_logits[0, mask_token_index, :]
#     print(mask_token_logits)
    top_k_tokens = torch.topk(mask_token_logits, k, dim=1).indices.numpy()
    _labels = labels[0, mask_token_index].numpy()
    _labels = np.vstack(_labels) #vertical stack to be able to compare to top_5_tokens
#     print(top_5_tokens)
#     print(_labels)
    
    for index, mask_index in enumerate(mask_token_index):
        begin_index = max(mask_index-verbose_length,0)
        begin_index_offset = max(mask_index-verbose_length,0) - (mask_index-verbose_length) # Will be 0 if no offset
        
        input_verbose_sequence = _input[0, max(mask_index-verbose_length,0):mask_index+verbose_length].clone()
        labeled_verbose_sequence = input_verbose_sequence.clone()
        
#         print(labeled_verbose_sequence)
        
        labeled_verbose_sequence[verbose_length-begin_index_offset] = labels[0, mask_index]
        
        print("Masked Input: ")
        print(f"\t{list(map(lambda x: tokenizer.decode([x]) , input_verbose_sequence))}")
#         print(f"\t{tokenizer.decode(input_verbose_sequence)}")
        print("Labeled Input: ")
        print(f"\t{list(map(lambda x: tokenizer.decode([x]) , labeled_verbose_sequence))}")
#         print(f"\t{tokenizer.decode(labeled_verbose_sequence)}")
        print("Predictions: ")
        for prediction in top_k_tokens[index]:
            prediction_verbose_sequence = input_verbose_sequence.clone()
            prediction_verbose_sequence[verbose_length-begin_index_offset] = prediction
            print(f"\t{list(map(lambda x: tokenizer.decode([x]) , prediction_verbose_sequence))}")
#             print(f"\t{tokenizer.decode(prediction_verbose_sequence)}")
        print("")
    
    correct_labels = np.any(top_k_tokens == _labels, axis=1)
    percent_correct = np.sum(correct_labels)/correct_labels.size
    print(f"Total Input Tokens: {_input.squeeze().shape}")
    print(f"Total Correct for Top {k}: {np.sum(correct_labels)}")
    print(f"Total Mask: {correct_labels.size}")
    print(f"Percent Correct: {percent_correct*100:.2f}%")
    return percent_correct

In [48]:
print(tokenized_input)
print(tokenized_input[0,0:200])
mask_token_index = torch.where(tokenized_input == tokenizer.mask_token_id)[1]
print(mask_token_index)
token_logits = model(_input)[0]
print(token_logits)
mask_token_logits = token_logits[0, mask_token_index, :]
print(mask_token_logits)
top_k_tokens = torch.topk(mask_token_logits, k=5, dim=1).indices.numpy()
print(top_k_tokens)
_labels = labels[0, mask_token_index].numpy()
_labels = np.vstack(_labels)
print(_labels)
for index, mask_index in enumerate(mask_token_index):
    input_verbose_sequence = tokenized_input[0, max(mask_index-6,0):mask_index+6].clone()
    print(input_verbose_sequence)
    break

tensor([[    2,     0,     9, 50708,     9,     4,     9,     4,     9,     4,
             9,   379,     9, 50708,     4, 11009,     9, 76615,     4,     9,
             4,     9, 17114,     9, 54855,     9,   882,     9,   379,     9,
             4,     9, 73189,     9, 15472,     4, 12358,     9,  6858,     9,
             4,     4,     4,    47,     3]])
tensor([    2,     0,     9, 50708,     9,     4,     9,     4,     9,     4,
            9,   379,     9, 50708,     4, 11009,     9, 76615,     4,     9,
            4,     9, 17114,     9, 54855,     9,   882,     9,   379,     9,
            4,     9, 73189,     9, 15472,     4, 12358,     9,  6858,     9,
            4,     4,     4,    47,     3])
tensor([ 5,  7,  9, 14, 18, 20, 30, 35, 40, 41, 42])
tensor([[[10.8335, -2.9528, 25.5903,  ..., -0.8487, -1.9001, -1.6345],
         [22.6757, -0.4105,  5.9939,  ...,  1.3035, -0.7734,  0.5439],
         [ 8.6228, -5.1675,  3.9214,  ..., -4.0427, -4.7083, -4.0134],
         ...,
  

In [51]:
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	['xxbos', 'xxunk', ' ', 'models', ' ', 'xxfld', ' ', 'xxfld', ' ', 'xxfld', ' ']
Labeled Input: 
	['xxbos', 'xxunk', ' ', 'models', ' ', 'are', ' ', 'xxfld', ' ', 'xxfld', ' ']
Predictions: 
	['xxbos', 'xxunk', ' ', 'models', ' ', 'they', ' ', 'xxfld', ' ', 'xxfld', ' ']
	['xxbos', 'xxunk', ' ', 'models', ' ', 'and', ' ', 'xxfld', ' ', 'xxfld', ' ']
	['xxbos', 'xxunk', ' ', 'models', ' ', 'we', ' ', 'xxfld', ' ', 'xxfld', ' ']
	['xxbos', 'xxunk', ' ', 'models', ' ', 'xxunk', ' ', 'xxfld', ' ', 'xxfld', ' ']
	['xxbos', 'xxunk', ' ', 'models', ' ', 'you', ' ', 'xxfld', ' ', 'xxfld', ' ']

Masked Input: 
	['xxunk', ' ', 'models', ' ', 'xxfld', ' ', 'xxfld', ' ', 'xxfld', ' ', 'the', ' ']
Labeled Input: 
	['xxunk', ' ', 'models', ' ', 'xxfld', ' ', 'xxunk', ' ', 'xxfld', ' ', 'the', ' ']
Predictions: 
	['xxunk', ' ', 'models', ' ', 'xxfld', ' ', 'xxunk', ' ', 'xxfld', ' ', 'the', ' ']
	['xxunk', ' ', 'models', ' ', 'xxfld', ' ', 'and', ' ', 'xxfld', ' ', 'the', ' ']
	['xxun

0.45454545454545453

# Trying Mask Filling on New Unseen Pantip Samples

### Test Tokenize on Pantip Sample ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ
https://pantip.com/topic/40006922

In [52]:
text = "ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ  ก่อนอื่นผมต้องบอกก่อนเลยว่าคนเราจะเลือกกินอาหารแบบไหนชอบแบบไหนเป็นเรื่องของความชอบส่วนตัวนะครับทุกคนมีสิทธิในการเลือกของที่ชอบและไม่ชอบอยู่แล้ว แต่ผมรู้สึกว่าตอนนี้ผมกำลังประสบปัญหาที่ดูเหมือนจะเล็กแต่กลายเป็นว่ามันค่อนข้างใหญ่ ผมคบกับแฟนมา6ปีแล้วครับ ผมเป็นคนชอบกินอาหารญี่ปุ่นและปลาดิบแต่แฟนผมไม่กินปลาดิบเลย ผมอยากกินบุฟเฟ่เนื้อแต่แฟนผมก็ไม่กินเนื้อ เราเลยไม่ได้เข้าทานร้านบุฟเฟ่เนื้อและบุฟเฟ่อาหารญี่ปุ่นกันเพราะรู้สึกลัวแฟนผมทานไม่คุ้ม และเรื่องใหญ่เลยคือผมเป็นคนชอบทานอาหารรสจัดและรสเผ็ดมาก แต่แฟนผมทานเผ็ดไม่ได้เลยเวลาเราไปกินส้มตำกันก็จะสั่ง ส้มตำไม่ใส่พริก ต้มแซ่บไม่ใส่พริก ลาบไม่ใส่พริก ร้านกับข้าวอื่นๆก็เช่นกันแฟนผมจะไม่ชอบกินผักไม่ค่อยสั่งกับข้าวที่เป็นผักแล้วผมชอบผักบุ้งทอดกรอบ เห็ดหอมสดทอดมาก แต่ก็ไม่ได้สั่งเพราะว่าเธอไม่กินถึงเค้าจะบอกให้สั่งเลยๆก็เถอะแต่ผมก็ยังเกรงใจเธออยู่ดีอ่ะครับ ผมรู้สึกกินอาหารไม่มีความสุขเลยชีวิตผมขาดรสเผ็ดไปเหมือนจะขาดใจเหมือนมันทำให้ขาดความสุขไปอย่างนึงเลยอ่ะครับ ยิ่งถ้าเราแต่งงานกันแล้วผมก็อาจจะต้องมีปัญหาเรื่องนี้มากขึ้น พอผมเห็นคู่ที่ชอบทานอาหารเหมือนๆกันเห็นเค้ากินอาหารกันอย่างมีความสุขแล้วผมรู้สึกอิจฉามากๆเลย มีใครเคยมีปัญหาแบบผมมั้ยครับแล้วจะแก้ปัญหานี้ยังไงดีครับ"
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.1)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'xxfld', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']
Labeled Input: 
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'แล้ว', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']
Predictions: 
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'แล้ว', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'และ', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', ' ', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'หรือ', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']
	['แฟน', 'ที่', 'กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'จน', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง']

Masked Input: 
	['กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'xxfld', 'รู้สึก', 'xxfld', 'ความสุข', 'ไป', 'อย่าง', 'นึง', 'บ้าง']
Labeled Input: 
	['กิน', 'อาหาร', 'ไม่ถูกปาก', 'กัน', 'xxfld', 'รู้สึก', 'เสีย', 'ความสุข', 'ไป', 'อย่

0.7560975609756098

### Test Tokenize on Pantip Sample อาการแบบนี้คือไรกัน?
https://pantip.com/topic/40009518

In [66]:
text = "อาการแบบนี้คือไรกัน?  เขาคุยกับเรามา 5-6 เดือน เราตามจีบเขานะคะ ก็คุยกันมา ในระยะเวลาเขาบอกว่า ถ้าเราลด นน เพื่อเขาได้ เขาจะยอมเป็นแฟนเรา ตรรกะโง่มากนะคะ แต่ถามว่าทำมั้ย ทำค่ะ พอไปรู้ว่าเขาคุยกับเพื่อน เพื่อนเขาถามว่า รู้สึกยังไงกับเรา เขาตอบเพื่อนว่า เขาว่าเขาควรอยู่คนเดียว ยังไม่พร้อมจะรักใคร จนตอนนี้เราเริ่มรู้สึกว่า ทำไมเราต้องทำขนาดนั้น ถ้าเขาจะรัก รักที่เป็นตัวเราไม่ได้หรอ หลังๆเลยเริ่มสนใจเขาน้อยลง แต่ยังคุยกันเหมือนเดิม เราลองแกล้งเงียบไป ไม่ทักไปครึ่งวัน ปกติเราจะมีการมอนิ่งกันตอนเช้าค่ะ พอเราไม่ทักไป เขาทำงานเสร็จ ถึงเวลาพักของเขา เขาก็ทักมาว่า กินข้าวกัน เราก็ยิ่ง งง ก็คิดว่า เขาอาจจะชินหับการคุยกับเราทุกวันเฉยๆ นี่เลยไม่ได้สนใจในส่วนนั้น เราก็ตอบตามปกติ จนเมื่อคืนมีคนมาทักเราจีบเรา จะไปส่งเราที่บ้าน เราก็เลยเล่าให้เขาฟังว่า ให้ไลน์ไป ให้เขาไปส่งอยู่แต่ไมไ่ด้นั่งรถคันเดียวกัน เราก็ขับของเรา คนที่มาจีบเราเขาก็ขับคันของเขาแค่มาส่งเฉยๆ พอเช้ามาเขาทักมามอนิ่ง ก็ถามเราเรื่องเมื่อคืน เราทำงานที่กลับดึกมากๆไม่ได้ทักไปบอกเขาไว้ว่า ถึงบ้านแล้วนะ เงียบไปทั้งคืนเลย เขาก็ถามเรื่องเมื่อคืนว่า หนุ่มไปส่งที่บ้านเป็นไงบ้าง ถามแต่เรื่องของผู้ชายที่มาจีบเราทั้งวัน จนเราเปลี่ยนเรื่องก็ยังกลับมาถามอีกรอบ ไออาการแบบนี้คืออะไรคะ ? ไหนเขาบอกอยากอยู่คนเดียว แต่พอเรามีคนเข้ามา ทำไมเขาถึงมีอาการแบบนี้ มาถามแบบนี้ซ้ำๆ คืออะไรกัน เราไม่อยากคิดอะไรไปเอง ใครพอจะตอบได้บ้างคะ ว่า ไอแบบนี้มันคืออะไร รู้สึกอะไรอยู่"
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.1)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', 'xxfld', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']
Labeled Input: 
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', '?', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']
Predictions: 
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', 'แน่', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', '?', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', 'หรือ', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', 'เนี่ย', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']
	['xxbos', 'อาการ', 'แบบนี้', 'คือ', 'ไร', 'กัน', '??', ' ', 'เขา', 'คุยกับ', 'เรา', 'มา']

Masked Input: 
	['เขา', 'คุยกับ', 'เรา', 'มา', 'xxnum', ' ', 'xxfld', ' ', 'เรา', 'ตามจีบ', 'เขา', 'นะคะ']
Labeled Input: 
	['เขา', 'คุยกับ', 'เรา', 'มา', 'xxnum', ' ', 'เดือน', ' ', 'เรา', 'ตามจีบ', 'เขา', 'นะคะ']
Predictions: 
	['เขา', 'คุยกับ', 'เรา', 'มา', 'xxnum', ' ', 'เดือน', ' ', 'เรา', 'ตามจีบ', 'เขา

0.9090909090909091

### Test Tokenize on Pantip Sample ร่างจำแลง (Literature)
https://pantip.com/topic/40039192

In [68]:
text = """ก้าวแรกของผมที่สัมผัสฝ่าเท้าลงภายในสถานเริงรมย์แห่งนั้น เสียงต่างๆรวมถึงภาพเคลื่อนไหวประดังประเดเข้าในโสตอย่างพร้อมเพรียง สร้างความอึกทึกมึนตึงไปหมด
แสงสีบนเพดานที่กระพริบกวัดแกว่งไปมาจากการควบคุมด้วยเทคโนโลยี สลับลำแสงท่ามกลางความมืดจนตาลายจำแนกวัตถุไม่ออก ยิ่งสำหรับคนที่ไม่เคยชินอย่างผมที่เพิ่งย่างกรายเข้ามาเป็นหนแรก
เสียงเพลงก็กระหึ่มกระแทกกระทั้นเร้าใจ กระตุ้นให้ทุกอณูส่วนของร่างกายอยากขยับเด้าดิ้นวาดลวดลายจนทรุดกองฮวบกับพื้น ดุจดั่งที่หลายคนบนพื้นเต้นกำลังทำอยู่ หรือมิเช่นนั้นก็หนวกหูรำคาญไปเลยอย่างที่ผมกำลังรู้สึก
ชัยนาทเห็นผมแต่ไกลได้อย่างไรไม่รู้ เขาคงคุ้นเคยกับห้องมืดสลัวที่มีแสงไฟเปลี่ยนสีตวัดกวัดไกวไปมาจนปวดตา จึงเห็นผมได้ไม่ยากเย็น"""
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.1)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	['xxfld', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']
Labeled Input: 
	['xxbos', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']
Predictions: 
	['xxbos', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']
	['เหมือนกับ', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']
	['พร้อมกับ', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']
	['เช่นเดียวกับ', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']
	['ดังเช่น', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส']

Masked Input: 
	['xxfld', 'ก้าวแรก', 'ของ', 'xxfld', 'ที่', 'สัมผัส', 'ฝ่าเท้า', 'ลง', 'ภายใน']
Labeled Input: 
	['xxfld', 'ก้าวแรก', 'ของ', 'ผม', 'ที่', 'สัมผัส', 'ฝ่าเท้า', 'ลง', 'ภายใน']
Predictions: 
	['xxfld', 'ก้าวแรก', 'ของ', 'ร่างกาย', 'ที่', 'สัมผัส', 'ฝ่าเท้า', 'ลง', 'ภายใน']
	['xxfld', 'ก้าวแรก', 'ของ', 'ผม', 'ที่', 'สัมผัส', 'ฝ่าเท้า', 'ลง', 'ภายใน']
	['xxfld', 'ก้าวแรก', 'ของ', 'ผู้', 'ที่', 'สัมผัส', 'ฝ่าเท้า', 'ลง', 'ภายใน']
	['xxfld', 'ก้าวแรก', 'ของ', 'ชีวิต', 'ที่', 'สัมผัส', 'ฝ่าเท้า', 'ลง', 'ภายใน']
	['xxfld', 'ก้าวแรก', 'ของ',

0.7333333333333333

### Test Tokenize on Own Sample

In [547]:
text = "สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว นี่คือการเว้นวรรคสองทีครับ จะได้ออกเป็นสอง"
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.15)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	ัสดีครับ<mask>ื่อไนท์ ตอนน
Labeled Input: 
	ัสดีครับ ผมชื่อไนท์ ตอนน
Predictions: 
	ัสดีครับ ผมชื่อไนท์ ตอนน
	ัสดีครับ เราชื่อไนท์ ตอนน
	ัสดีครับ เพื่อไนท์ ตอนน
	ัสดีครับ ชื่อไนท์ ตอนน
	ัสดีครับ เมื่อไนท์ ตอนน

Masked Input: 
	้องไปโรงเรียนแล้<mask> น<mask>คือการ
Labeled Input: 
	้องไปโรงเรียนแล้ว น<mask>คือการ
Predictions: 
	้องไปโรงเรียนแล้ว น<mask>คือการ
	้องไปโรงเรียนแล้วนะ น<mask>คือการ
	้องไปโรงเรียนแล้วผม น<mask>คือการ
	้องไปโรงเรียนแล้วว น<mask>คือการ
	้องไปโรงเรียนแล้ววว น<mask>คือการ

Masked Input: 
	โรงเรียนแล้<mask> น<mask>คือการ<mask>้
Labeled Input: 
	โรงเรียนแล้<mask> นี่คือการ<mask>้
Predictions: 
	โรงเรียนแล้<mask> นี่คือการ<mask>้
	โรงเรียนแล้<mask> นี้คือการ<mask>้
	โรงเรียนแล้<mask> นีคือการ<mask>้
	โรงเรียนแล้<mask> นิคือการ<mask>้
	โรงเรียนแล้<mask> น็คือการ<mask>้

Masked Input: 
	<mask> น<mask>คือการ<mask>้นวรรคสองที
Labeled Input: 
	<mask> น<mask>คือการเว้นวรรคสองที
Predictions: 
	<mask> น<mask>คือการเว้นวรรคสองที
	<mask> น<mask>คือการลดต้นวรรคส

0.8571428571428571

Or trying the Vanilla way to generate

In [15]:
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."

In [19]:
_input = tokenizer.encode(sequence, return_tensors="pt")
_input

tensor([[    0,    40,  2744,   789,  9774, 17624,    87,  6344,  2352,  1848,
           522, 13701,  1207, 17624,    87, 19898,   584,  1164,   721,    18,
           225,    57, 19100,  1207,    81, 12140,    73,   977,  1572,  1207,
           676,   633,  3789, 12847, 10865,   731, 11955, 15051,    84,     4,
         14855,  5963,    70,   524,   655,  8489,  3220,  3449,    18,     2]])

In [53]:
tokenizer.encode_plus(sequence)

{'input_ids': [0, 40, 2744, 789, 9774, 17624, 87, 6344, 2352, 1848, 522, 13701, 1207, 17624, 87, 19898, 584, 1164, 721, 18, 225, 57, 19100, 1207, 81, 12140, 73, 977, 1572, 1207, 676, 633, 3789, 12847, 10865, 731, 11955, 15051, 84, 4, 14855, 5963, 70, 524, 655, 8489, 3220, 3449, 18, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
mask_token_index = torch.where(_input == tokenizer.mask_token_id)[1]
mask_token_index

tensor([39])

In [22]:
token_logits = model(_input)[0]
token_logits

tensor([[[ -6.8569, -11.4544,  -6.2892,  ...,  -3.3654,  -1.7394,  -3.8599],
         [ -6.7165,  -7.8614, -10.1110,  ...,  -0.2116,  -1.2954,  -8.0255],
         [ -6.4925,  -9.5634,  -9.8982,  ...,  -3.4279,  -1.0313,  -6.8084],
         ...,
         [ -6.2978,  -6.5940,  -9.4618,  ...,   0.5986,  -2.2195,  -8.1321],
         [ -6.3295,  -9.3629,  -9.6262,  ...,   0.0172,  -8.5017,  -5.1815],
         [ -6.8569, -11.4543,  -6.2891,  ...,  -3.3655,  -1.7393,  -3.8597]]],
       grad_fn=<AddBackward0>)

In [23]:
mask_token_logits = token_logits[0, mask_token_index, :]
mask_token_logits

tensor([[ -6.0691,  -7.5146, -11.4188,  ...,   3.1234,  -1.2106,  -6.2130]],
       grad_fn=<IndexBackward>)

In [24]:
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
top_5_tokens

[1228, 1187, 1796, 18, 12549]

In [25]:
for token in top_5_tokens:
     print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help ed our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help  in our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help  to our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help . our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help  her our carbon footprint.


# Testing Text-Generation

In [26]:
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [27]:
text_gen("As far as I am concerned, I will", max_length=50, do_sample=False)

NotImplementedError: Generation is currently not supported for RobertaForMaskedLM. Please select a model from ['XLNetLMHeadModel', 'TransfoXLLMHeadModel', 'ReformerModelWithLMHead', 'GPT2LMHeadModel', 'OpenAIGPTLMHeadModel', 'CTRLLMHeadModel', 'TFXLNetLMHeadModel', 'TFTransfoXLLMHeadModel', 'TFGPT2LMHeadModel', 'TFOpenAIGPTLMHeadModel', 'TFCTRLLMHeadModel'] for generation.

In [281]:
prompt = "สวัสดีครับ"
_inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
print(_inputs)
[tokenizer.decode([tok]) for tok in _inputs[0]]

tensor([[1783,  275,  566,  278,  333,  275,  282]])


['สว', 'ั', 'สด', 'ี', 'คร', 'ั', 'บ']

In [55]:
# def _text_generation(prompt, max_length=250)
prompt = "สวัสดีครับ"

def _add_mask_to_last(prompt, tokenizer, device=None):
    # Tokenize the sequence with special tokens inserted, remove last token and replace with mask
    if type(prompt) is str:
        _input = tokenizer.encode(prompt)
        _input[0, -1] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
        _temp_tensor = torch.zeros((_input.shape[0], 1), dtype=torch.int64)
        _temp_tensor[0][0] = tokenizer.sep_token_id
        return torch.cat((_input, _temp_tensor), dim=1)
    else: # if it is tensor
        _input = prompt.clone()
        _temp_tensor = torch.zeros((_input.shape[0], 2), dtype=torch.int64)
        _temp_tensor[0][0] = tokenizer.mask_token_id
        _temp_tensor[0][1] = tokenizer.sep_token_id
#         _temp_tensor = _temp_tensor.to(device)
        return torch.cat((prompt, _temp_tensor), dim=1)
_add_mask_to_last(prompt, tokenizer)

tensor([[  2, 313,  15,   4,   3]])

In [56]:
tokenizer.sep_token_id

3

In [490]:
device = torch.device("cuda:0")

In [537]:
cpu_device = torch.device("cpu")
model.to(cpu_device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(20000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [69]:
def _predict_last_token(_input, tokenizer, k, top_p):
    """top_k – (optional) int The number of highest probability vocabulary tokens to keep for top-k-filtering. Between 1 and infinity. Default to 50.

    top_p – (optional) float The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling. Must be between 0 and 1. Default to 1."""
    mask_token_index = torch.where(_input == tokenizer.mask_token_id)[1]
#     _input.to(device)
    token_logits = model(_input)[0]
    mask_token_logits = token_logits[0, mask_token_index, :]
    _mask_token_logits_cpu = mask_token_logits.clone().cpu()
    top_k_tokens = torch.topk(_mask_token_logits_cpu, k, dim=1).indices[0].numpy()
    if np.random.random() < top_p:
        return top_k_tokens[0]
    else:
        return top_k_tokens[1:][np.random.randint(low=0, high=top_k_tokens.size-1)]

In [73]:
def _text_generate(prompt, tokenizer, length = 250, top_k=10, top_p=0.95, device=None):
#     _input = _add_mask_to_last(prompt, tokenizer)
    _input = prompt
    for i in range(length):
        _input = _add_mask_to_last(_input, tokenizer, device=device)
#         _input = _input.to(device)
        _input[0, -2] = _predict_last_token(_input, tokenizer, k=top_k, top_p = top_p)
        _input = _input[:, :-1].clone()
    print(f"{list(map(lambda x: tokenizer.decode([x]) , _input[0]))}")
#     print(tokenizer.decode(_input[0]))

In [510]:
_input.to(device)

tensor([[   0, 1783,  275,  566,  278,  333,  275,  282,    4]],
       device='cuda:0')

In [77]:
_text_generate("""การตั้งกระทู้และการเชียร์บอล รวมถึงกิจกรรมทั้งหมดของชมรม ฯ เป็นไปเพื่อความบันเทิงของสมาชิกทุกๆท่าน""", tokenizer,length = 200,top_k=8, top_p=0.85)

['xxbos', 'การ', 'ตั้งกระทู้', 'และ', 'การ', 'เชียร์', 'บอล', ' ', 'รวมถึง', 'กิจกรรม', 'ทั้งหมด', 'ของ', 'ชมรม', ' ', 'ฯ', ' ', 'เป็นไป', 'เพื่อ', 'ความบันเทิง', 'ของ', 'สมาชิก', 'ทุกๆ', 'ท่าน', 'ทั้ง', ' ', 'xxnum', ' ', 'ท่าน', ' ', 'และ', 'ที่', ' ', 'xxnum', ' ', 'กิจกรรม', 'การ', 'ตั้งกระทู้', '+', 'การ', 'เชียร์', 'บอล', ' ', 'กิจกรรม', 'การ', 'ตั้งกระทู้', '+', 'การ', 'เชียร์', 'หงส์', 'ปาร์ตี้', ' ', 'และ', 'กิจกรรม', 'การ', 'เชียร์', 'หงส์', 'ปาร์ตี้', ' ', 'และ', 'เครื่องดื่ม', 'ต่าง ๆ', ' ', 'กิจกรรม', 'การ', 'ตั้งกระทู้', '+', 'การ', 'เชียร์', 'หงส์', 'สังสรรค์', 'ปาร์ตี้', ' ', 'และ', 'เครื่องดื่ม', 'ต่าง ๆ', ' ', 'กิจกรรม', 'การ', 'ตั้งกระทู้', '+', 'การ', 'เชียร์', 'หงส์', 'ปาร์ตี้', 'เครื่องดื่ม', 'ต่าง ๆ', ' ', 'สำหรับ', 'กิจกรรม', 'การ', 'ตั้งกระทู้', '+', 'การ', 'เชียร์', 'หงส์', 'ปาร์ตี้', ' ', 'เครื่องดื่ม', 'ต่าง ๆ', ' ', 'นอกจากนี้', 'กิจกรรม', 'การ', 'ตั้งกระทู้', '+', 'เป็นการ', 'เชียร์', 'หงส์', 'ปาร์ตี้', ' ', 'และ', 'เครื่องดื่ม', 'ต่าง ๆ', 'ที่', 'กิจกรรม'

In [76]:
_text_generate("""ตอนนี้เป็นเวลา""", tokenizer,length = 200,top_k=8, top_p=0.85)

['xxbos', 'ตอนนี้', 'เป็นเวลา', 'ที่', ' ', 'xxnum', ' ', 'แล้ว', 'นะ', ' ', '\n', ' ', 'แต่', 'ถ้า', 'มี', 'เพื่อน', 'ที่', ' ', 'xxnum', ' ', '\n', ' ', 'แล้ว', ' ', '\n', ' ', 'ก็', 'จะ', 'มี', 'เพื่อน', 'ที่', ' ', 'xxnum', ' ', '\n', ' ', 'แล้ว', ' ', 'ตอนนี้', 'กินเวลา', 'ได้', ' ', 'xxnum', '}', 'เลย', ' ', '\n', ' ', 'แต่', 'ถ้า', 'มี', 'เพื่อน', 'ที่', ' ', 'xxnum', ' ', 'xxnum', ' ', '\n', ' ', 'แล้ว', ' ', 'ตอนนี้', 'กินเวลา', 'ได้', ' ', 'xxnum', '}', 'เลย', ' ', '\n', 'หมายเหตุ', 'ถ้า', 'มี', 'เพื่อน', 'ที่', ' ', 'xxnum', ' ', '\n', ' ', 'แล้ว', ' ', 'ตอนนี้', 'กินเวลา', 'ได้', ' ', 'xxnum', '}', ' ', '\n', 'หมายเหตุ', 'หมายเหตุ', ':', ' ', '\n', ' ', 'ถ้า', 'มี', 'เพื่อน', 'ที่', ' ', 'xxnum', ' ', '/', ' ', 'xxnum', ' ', '\n', ');', 'ใช้เวลานาน', 'ได้', ' ', 'xxnum', '}', 'เลย', ' ', 'ได้', ' ', 'xxnum', ' ', '\n', ' ', 'ถ้า', 'มี', 'เพื่อน', 'ที่', ' ', 'xxnum', ' ', 'และ', 'xxnum', ' ', 'ใช้เวลานาน', 'ได้', ' ', 'xxnum', '}', ' ', '\n', 'คำถาม', 'หมายเหตุ', ':', 'หมาย

In [308]:
top_k_tokens.size

10

In [306]:
np.random.random()

0.48881225040001264

In [43]:
prompt_length = len(tokenizer.decode(_inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
prompt_length

10

In [282]:
outputs = model.generate(_inputs, max_length=250, do_sample=True, top_p=0.95, top_k=5)
outputs

tensor([[1783,  275,  566,  278,  333,  275,  282,  282,  282,  282,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  

In [34]:
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]
generated

'สวัสดีครับ                                                                                                                                                                                                                                                   '

In [46]:
tokenizer.decode(outputs[0])

'สวัสดีครับบบบบบbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb'